### 1. Преподготовка датасета

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [120]:
df_train = pd.read_csv('train.csv')

In [121]:
len(df_train)

6261

In [122]:
df_train.head(2)

,Unnamed: 0,id,atm_group,address,address_rus,lat,long,target
0,0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.01150
1,1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.02971


In [123]:
#420 банкоматов без координат.
df_train[df_train['lat'].isnull()].count()

Unnamed: 0     304
id             304
atm_group      304
address        304
address_rus      0
lat              0
long             0
target         304
dtype: int64

In [124]:
#Пока удалю иx
df_train = df_train[~(df_train['lat'].isnull())]

In [125]:
#переименнновываю широту
df_train = df_train.rename(columns={'long':'lon'})

In [126]:
df_train.drop('Unnamed: 0', axis=1, inplace = True)

In [127]:
#предобразовываю в геодатафрейм
df_train = gpd.GeoDataFrame(df_train, geometry=gpd.points_from_xy(df_train.lon, df_train.lat))

In [128]:
df_train.geometry = df_train.geometry.set_crs('EPSG:4326').to_crs('EPSG:3857')

In [129]:
df_train.geometry.head()

0    POINT (15889556.988 5932448.267)
1    POINT (15891230.008 5931854.476)
2    POINT (15889868.014 5935451.755)
3     POINT (4176166.950 7519858.012)
4    POINT (9819628.983 10858343.208)
Name: geometry, dtype: geometry

### 2. Преподготовка объектов OSM

In [63]:
df_objects = pd.read_csv('all_object.csv')

In [65]:
len(df_objects)

5846502

In [66]:
df_objects.head(2)

,bulding_type,geometry,lon,lat
0,retail,POINT (37.53998433511941 55.91601868840449),37.539984,55.916019
1,retail,POINT (37.431339550000004 55.84574095000001),37.431340,55.845741


In [67]:
 df_objects['geometry'][:1]

0    POINT (37.53998433511941 55.91601868840449)
Name: geometry, dtype: object

In [68]:
df_objects = gpd.GeoDataFrame(df_objects, geometry=gpd.points_from_xy(df_objects.lon, df_objects.lat))

In [69]:
df_objects.geometry = df_objects.geometry.set_crs('EPSG:4326').to_crs('EPSG:3857')

### 3. Создание фичей

In [130]:
objects = df_objects.bulding_type.unique()

In [131]:
for i in objects:
  col_name = 'distance_to_'+i
  df_temp = df_train[['id','geometry']].\
  sjoin_nearest(df_objects[df_objects['bulding_type']==i], how = 'inner', max_distance=10000, distance_col='distance').\
  groupby('id').distance.min().reset_index()
  df_temp.rename(columns = {'distance':col_name}, inplace = True)
  df_train=df_train.merge(df_temp, how='left', on = 'id')

In [132]:
df_train.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5957 entries, 0 to 5956
Data columns (total 45 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   id                               5957 non-null   float64 
 1   atm_group                        5957 non-null   float64 
 2   address                          5957 non-null   object  
 3   address_rus                      5957 non-null   object  
 4   lat                              5957 non-null   float64 
 5   lon                              5957 non-null   float64 
 6   target                           5957 non-null   float64 
 7   geometry                         5957 non-null   geometry
 8   distance_to_retail               5672 non-null   float64 
 9   distance_to_apartments           5826 non-null   float64 
 10  distance_to_commercial           5627 non-null   float64 
 11  distance_to_office               5061 non-null   float64 
 12

In [133]:
df_train.head()

,id,atm_group,address,address_rus,lat,lon,target,geometry,distance_to_retail,distance_to_apartments,...,distance_to_alfa_atm,distance_to_ros_atm,distance_to_vending_parking,distance_to_bus_stop,distance_to_railway_station,distance_to_tram_stop,distance_to_railway_halt,distance_to_airport,distance_to_parking,distance_to_parking_underground
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,POINT (15889556.988 5932448.267),7.538905,216.343704,...,NaN,NaN,NaN,187.914499,2888.292033,NaN,1222.350315,NaN,2892.577916,2346.129844
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,POINT (15891230.008 5931854.476),272.351877,9.606115,...,NaN,NaN,NaN,129.723638,3143.092355,NaN,2995.399455,NaN,2767.793411,2124.799118
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,POINT (15889868.014 5935451.755),428.158970,177.741156,...,NaN,NaN,NaN,175.015423,1734.715409,NaN,2398.879500,NaN,1815.884777,1864.222802
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,POINT (4176166.950 7519858.012),6.220610,208.837730,...,193.034224,56.762432,537.06515,83.760176,111.400631,705.043782,2193.533083,NaN,106.652808,471.901686
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,POINT (9819628.983 10858343.208),367.920008,259.466732,...,NaN,NaN,NaN,151.732098,9329.940868,NaN,NaN,NaN,5763.343722,5842.299373


In [134]:
df_train.to_csv('data_with_OSM', index = False)